In [1]:
!pip install be-great datasets transformers scikit-learn

In [2]:
import pandas as pd
from be_great import GReaT

In [3]:
data = pd.read_csv("/content/drive/MyDrive/diabetes.csv")

In [5]:
model = GReaT(llm='distilgpt2', batch_size=32, epochs=30, fp16=False)  # Adjust epochs/batch_size as needed

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
model.fit(data)

/usr/local/lib/python3.12/dist-packages/be_great/great.py:174: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `GReaTTrainer.__init__`. Use `processing_class` instead.
  great_trainer = GReaTTrainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,0.947900


In [7]:
synthetic_data = model.sample(n_samples=100)

  0%|          | 0/100 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
187it [00:01, 100.46it/s]


In [8]:
print("Synthetic data preview:")
print(synthetic_data.head())

Synthetic data preview:
   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0          6.0    116.0           74.0            0.0      0.0  23.2   
1          7.0    147.0           62.0            0.0      0.0  45.4   
2          0.0    103.0           70.0            0.0      0.0  29.5   
3          1.0    111.0           70.0            0.0      0.0  30.4   
4          5.0    103.0           82.0            0.0     87.0  37.0   

   DiabetesPedigreeFunction   Age  Outcome  
0                     0.842  24.0      0.0  
1                     0.839  22.0      0.0  
2                     0.256  31.0      0.0  
3                     0.238  44.0      0.0  
4                     0.531  21.0      0.0  


In [10]:
from google.colab import files

synthetic_data.to_csv('synthetic_data.csv', index=False)
files.download('synthetic_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
!pip install sdmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.4/198.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 5.9 MB/s eta 0:00:00


In [13]:
from sdmetrics.single_table import CSTest, ContinuousKLDivergence, DiscreteKLDivergence
from sdmetrics.reports.single_table import QualityReport

In [18]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Exact Match Count: number of identical rows in both datasets
def exact_match_count(real_df, synth_df):
    merged = pd.merge(real_df, synth_df, how='inner')
    return len(merged)

# Nearest Neighbor Distance: average distance from synthetic records to closest real record
def nearest_neighbor_distance(real_df, synth_df):
    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(real_df.values)
    distances, _ = nn.kneighbors(synth_df.values)
    return np.mean(distances)

# Simple Membership Inference Attack Simulation (based on nearest neighbor threshold)
def membership_inference_risk(real_df, synth_df, threshold=0.01):
    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(real_df.values)
    distances, _ = nn.kneighbors(synth_df.values)
    # Fraction of synthetic samples very close to real samples (within threshold)
    risk = np.mean(distances < threshold)
    return risk

# Example usage
exact_matches = exact_match_count(data, synthetic_data)
print(f"Exact Match Count: {exact_matches}")

nn_distance = nearest_neighbor_distance(data, synthetic_data)
print(f"Average Nearest Neighbor Distance: {nn_distance:.4f}")

membership_risk = membership_inference_risk(data, synthetic_data, threshold=0.05)
print(f"Membership Inference Risk: {membership_risk:.4f}")


Exact Match Count: 0
Average Nearest Neighbor Distance: 10.8720
Membership Inference Risk: 0.0000
